完成从 Xsens txt 文件到 OpenSim .sto 文件转换

In [1]:
# 查看包含四元数的 .sto 数据
import os
filename = "/Users/wangshuaibo/Library/CloudStorage/OneDrive-bsu.edu.cn/Archive/代码_OpenSense/OpenSense_Python/OpenSenseExample/OpenSenseExample_Modified/MT_012005D6_009-001_orientations.sto"

with open(filename, "r") as f:
    count = 0
    for line in f:
        count +=1
        if "enddata" in line:
            metadata_row_num = count

    metadata = []   
    count = 0
    for line in f:
        count += 1
        if count<metadata_row_num:
            print(line)
            metadata.append(line)
        
        if count==7:
            test_data= line
    
total_cout = count
print(f"文件共包含 {total_cout} 行")
print(f"头文件包含在前 {metadata_row_num} 行")
print(f"头文件为：{metadata}")

# 验证 sto 中的数据类型
print(f".sto中的内容同样为字符串{type(test_data[0])}")
# 查看 sto 中的数据格式
print(f".sto 的格式为:\n{repr(test_data)}")


文件共包含 0 行


NameError: name 'metadata_row_num' is not defined

In [2]:
# 定义演示数据
import pandas as pd
columns = ["time"] + [segment + quat for segment in ["Pelvis","Leg","Arm"] for quat in ["_q0","_q1","_q2","_q3"]]
df = pd.DataFrame([[0,1,2,3,4,5,6,7,8,9,10,11,12],
                   [0,2,2,3,4,5,6,7,8,9,10,11,12]],
                  columns=columns)
# 定义一个函数，用于将每个分组的数据转换为字符串列表
def join_quaternions(group):
    return ','.join(map(str, group.values.tolist()[0]))

# 按 segment 分组
pelvis_group = df[columns[:4]]
leg_group = df[columns[4:8]]
arm_group = df[columns[8:]]

# 将每个分组的数据转换为字符串列表
pelvis_string = join_quaternions(pelvis_group)
leg_string = join_quaternions(leg_group)
arm_string = join_quaternions(arm_group)

# 将所有分组的字符串用 `\t` 连接
result = f"{pelvis_string}\t{leg_string}\t{arm_string}\n"
print(repr(result))
type(pelvis_string)


'0,1,2,3\t4,5,6,7\t8,9,10,11,12\n'


str

In [3]:
# 转换为 opensim 格式
line_len = (df.shape[1]+1) * 2

def line_df_to_sto(line_len,row=0):
    line = [None] * line_len
    for i in range(line_len):
        # 此处也需要修改
        line[0] = df.iloc[row,0]
        if i%2 != 0 and (i-1)%8 != 0:
            line[i] = ","
        elif (i-1)%8 == 0:
            line[i] = "\t"
        elif i != 0 :
            #print(i)
            line[i] = df.iloc[row,int(i/2-1)]

    # 将列表转换为字符串
    result = ""
    for n in line:
        result += str(n)
    return result.strip("\t") + "\n"


In [4]:
harzt = 100
# 将hartz 填入表头
metadata = f'''DataRate={harzt:.6f}
DataType=Quaternion
version=3
OpenSimVersion=4.4-2022-10-11-798caa8
endheader'''

header_list = ["time"] + [segment+"_imu" for segment in ["Pelvis","Leg","Arm"]]
header_str = "\t".join(header_list) + "\n"


In [5]:
with open("test_quat.sto", "w") as f:
    # 先写属性信息
    f.write(metadata)
    f.write(header_str)
    for j in range(df.shape[0]):
        sto_line = line_df_to_sto(line_len,row=j)
        f.write(sto_line)
    